In [1]:
import pandas as pd
import time

Read the file

In [2]:
# df = pd.read_csv('adult.csv')
df = pd.read_csv('MOCK_DATA1.csv')
# df = pd.read_csv('MOCK_DATA_4k.csv')
# df = pd.read_csv('MOCK_DATA1_6k.csv')
# df = pd.read_csv('MOCK_DATA1_8k.csv')
# df = pd.read_csv('MOCK_DATA1_12k.csv')



Rows

In [3]:
# df.shape

In [4]:
# df.info()

DF Correlation

In [5]:
# Select only numeric columns for correlation calculation
# numeric_df = df.select_dtypes(include=['number'])

# Calculate correlation matrix
# correlation_matrix = numeric_df.corr()
# print(correlation_matrix)

In [6]:
# import pandas as pd

def anonymize_sensitive_fields(df):
    # ----------------------
    # Suppress Direct Identifiers by masking
    # ----------------------
    # Mask First Name and Last Name
    if 'first_name' in df.columns:
        df['first_name'] = '*'

    if 'last_name' in df.columns:
        df['last_name'] = '*'

    # Age: generalize into custom bins
    if 'Age' in df.columns or 'age' in df.columns:
        df['age'] = pd.cut(df['age'],
                       bins=[0, 20, 30, 40, 50, 60, 100],
                       labels=["0-20", "21-30", "31-40", "41-50", "51-60", "60+"])

    if 'email' in df.columns:
        def mask_email(email):
            if isinstance(email, str) and "@" in email:
                domain = email.split('@')[1]
                domain_suffix = domain.split('.')[-1] if '.' in domain else ''
                return "****@****." + domain_suffix
            return '*'
        df['email'] = df['email'].apply(mask_email)

    if 'Credit_Card' in df.columns:
        df['Credit_Card'] = '*'

    if 'CVV' in df.columns:
        df['CVV'] = '*'

    if 'Card Date-of-Expiry' in df.columns:
        df['Card Date-of-Expiry'] = '*'

    # ----------------------
    # Generalize Quasi-identifiers
    # ----------------------

    # Ethnicity & Race: Generalize into broader groups (example)
    # if 'Ethnicity' in df.columns:
    #     df['Ethnicity'] = df['Ethnicity'].replace({
    #         'Chinese': 'Asian',
    #         'Indian': 'Asian',
    #         'Malay': 'Asian',
    #         'White': 'Caucasian',
    #         'Black': 'African',
    #         'Hispanic': 'Latino',
    #     }).fillna('Other')

    # if 'Race' in df.columns:
    #     df['Race'] = df['Race'].replace({
    #         'Chinese': 'Asian',
    #         'Indian': 'Asian',
    #         'Malay': 'Asian',
    #         'White': 'Caucasian',
    #         'Black': 'African',
    #         'Hispanic': 'Latino',
    #     }).fillna('Other')

    # Occupation: Generalize to industry or first word
    # if 'occupation' in df.columns:
    #     df['occupation'] = df['occupation'].apply(lambda x: x.split()[0] if isinstance(x, str) else 'Other')

    # Education Level: Generalize to broad categories
    if 'Education Level' in df.columns:
        df['Education Level'] = df['Education Level'].replace({
            'High School': 'Secondary',
            'Diploma': 'Tertiary',
            'Bachelor': 'Tertiary',
            'Master': 'Postgraduate',
            'PhD': 'Postgraduate',
        }).fillna('Other')

    # Geolocation (Coordinates): Generalize to region (e.g., extract city)
    if 'Location' in df.columns:
        df['Location'] = df['Location'].apply(lambda x: x.split(',')[0] if isinstance(x, str) and ',' in x else 'Region')

    # Salary: Generalize into ranges
    # Salary: generalize into custom bins
    if 'Salary' in df.columns or 'salary' in df.columns:
        df['salary'] = pd.cut(df['salary'],
                          bins=[0, 20000, 40000, 60000, 80000, 100000, 9999999],
                          labels=["<20K", "20-40K", "40-60K", "60-80K", "80-100K", "100K+"])

    # Mask Phone Number
    if 'phone_number' in df.columns:
        def mask_phone(phone):
            if isinstance(phone, str) and len(phone) >= 4:
                return '*' * (len(phone) - 4) + phone[-4:]
            return '*'
        df['phone_number'] = df['phone_number'].apply(mask_phone)

    if 'longitude' in df.columns:
        df['longitude'] = df['longitude'].round(1)  # round to ~11km precision
    
    if 'latitude' in df.columns:
        df['latitude'] = df['latitude'].round(1)  # round to ~11km precision

    if 'street_address' in df.columns:
        df['street_address'] = '*'



    return df


In [14]:
# df = pd.read_csv("your_data.csv")
start = time.time()  # Start time
anonymized_df = anonymize_sensitive_fields(df)
shuffled_df = anonymized_df.sample(frac=1).reset_index(drop=True)
end = time.time()  # End time
# anonymized_df.to_csv("anonymized_output.csv", index=False)
elapsed_ms = (end - start) * 1000  # Convert seconds to milliseconds
print(f"Elapsed time: {elapsed_ms:.3f} ms")

shuffled_df

Elapsed time: 2.273 ms


,first_name,last_name,email,gender,Credit_Card,card_type,money,bank_name,occupation,race,university,latitude,longtidue,Country,street_address
0,*,*,****@****.com,Female,*,jcb,$177120.28,FIRST NATIONAL BANK,Research Associate,Colombian,Universitas Kristen Petra,-8.5,118.536023,Indonesia,*
1,*,*,****@****.com,Female,*,diners-club-carte-blanche,$145255.54,"Banco de Depósitos, S.A.",Senior Developer,Argentinian,Sri Lanka Institute of Information Technology ...,7.0,79.931095,Sri Lanka,*
2,*,*,****@****.com,Male,*,switch,$574834.70,RENASANT BANK,Recruiting Manager,Indonesian,Yunnan University,38.5,-122.804952,China,*
3,*,*,****@****.com,Female,*,jcb,$300511.84,Caisse régionale de crédit agricole mutuel Atl...,Senior Developer,Paraguayan,Universidad de Los Llanos,8.8,-76.240381,Colombia,*
4,*,*,****@****.cz,Female,*,jcb,$194792.06,DEUTSCHE BANK S.P.A.,Media Manager II,Pakistani,Universidad de Matanzas Camilo Cienfuegos,23.0,-82.385867,Cuba,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,*,*,****@****.uk,Male,*,mastercard,$436610.97,PEOPLES BANK,VP Product Management,Sri Lankan,Prague College,49.9,14.381846,Czech Republic,*
996,*,*,****@****.gov,Male,*,jcb,$515000.77,FIRST NATIONAL BANK,Help Desk Operator,Filipino,Universidad Anáhuac del Sur,19.4,-99.153295,Mexico,*
997,*,*,****@****.jp,Female,*,jcb,$26828.11,BANCA NAZIONALE DEL LAVORO S.P.A. (IN FORMA CO...,Quality Control Specialist,Latin American Indian,Ecole Nationale Supérieur de Géologie de Nancy,48.6,2.406265,France,*
998,*,*,****@****.net,Female,*,jcb,$377094.54,Caisse de crédit municipal de Nice,Quality Engineer,Costa Rican,Beijing Forestry University,41.2,-96.396605,China,*
